In [1]:
import numpy as np
import pandas as pd

# Anime Small

In [ ]:
anime_df = pd.read_csv("data/anime_small/rating.csv")
anime_df.columns = ["user_id", "item_id", "rating"]
anime_df = anime_df[anime_df["rating"] != -1]
print("Nr. of ratings: %d" % len(anime_df))
print("Nr. of users: %d" % anime_df["user_id"].nunique())
print("Nr. of items: %d" % anime_df["item_id"].nunique())
print("Density: %f" % (len(anime_df) / (anime_df["user_id"].nunique() * anime_df["item_id"].nunique())))
print("Avg. nr. of ratings per user: %f" % anime_df.groupby("user_id").size().mean())
print("Avg. nr. of ratings per item: %f" % anime_df.groupby("item_id").size().mean())

In [ ]:
profile_sizes = anime_df.groupby("user_id").size()
relevant_users = profile_sizes[profile_sizes >= 20].index.tolist()
anime_df = anime_df[anime_df["user_id"].isin(relevant_users)]

#item_popularity = anime_df.groupby("item_id").size()
#relevant_items = item_popularity[item_popularity >= 50].index.tolist()
#anime_df = anime_df[anime_df["item_id"].isin(relevant_items)]

In [ ]:
print("Nr. of ratings: %d" % len(anime_df))
print("Nr. of users: %d" % anime_df["user_id"].nunique())
print("Nr. of items: %d" % anime_df["item_id"].nunique())
print("Density: %f" % (len(anime_df) / (anime_df["user_id"].nunique() * anime_df["item_id"].nunique())))
print("Avg. nr. of ratings per user: %f" % anime_df.groupby("user_id").size().mean())
print("Avg. nr. of ratings per item: %f" % anime_df.groupby("item_id").size().mean())

In [ ]:
user_sample = np.random.choice(anime_df["user_id"].unique(), size=5000, replace=False)
df = anime_df[anime_df["user_id"].isin(user_sample)]
print("Nr. of ratings: %d" % len(df))
print("Nr. of users: %d" % df["user_id"].nunique())
print("Nr. of items: %d" % df["item_id"].nunique())
print("Density: %f" % (len(df) / (df["user_id"].nunique() * df["item_id"].nunique())))
print("Avg. nr. of ratings per user: %f" % df.groupby("user_id").size().mean())
print("Avg. nr. of ratings per item: %f" % df.groupby("item_id").size().mean())

In [ ]:
df.to_csv("data/anime_small/sample.csv", sep=";", header=False, index=None)

# Jester

In [ ]:
df1 = pd.read_excel("data/jester/jester-data-1.xls", header=None)
df1.drop(columns=[0], inplace=True)
df1 = df1.stack().reset_index()
df1.columns = ["user_id", "item_id", "rating"]

df2 = pd.read_excel("data/jester/jester-data-2.xls", header=None)
df2.drop(columns=[0], inplace=True)
df2 = df2.stack().reset_index()
df2.columns = ["user_id", "item_id", "rating"]

df3 = pd.read_excel("data/jester/jester-data-3.xls", header=None)
df3.drop(columns=[0], inplace=True)
df3 = df3.stack().reset_index()
df3.columns = ["user_id", "item_id", "rating"]

In [ ]:
n_users_1 = df1["user_id"].nunique()
n_users_2 = df2["user_id"].nunique()
n_users_3 = df3["user_id"].nunique()

user_mapping_1 = {b: a for a, b in enumerate(range(n_users_1))}
user_mapping_2 = {b: a+n_users_1 for a, b in enumerate(range(n_users_2))}
user_mapping_3 = {b: a+n_users_1+n_users_2 for a, b in enumerate(range(n_users_3))}

df1["user_id"] = df1["user_id"].map(user_mapping_1)
df2["user_id"] = df2["user_id"].map(user_mapping_2)
df3["user_id"] = df3["user_id"].map(user_mapping_3)
jester_df = df1.append(df2).append(df3)

jester_df["item_id"] = jester_df["item_id"].apply(lambda v: v-1)
jester_df.astype({"user_id": int, "item_id": int});

jester_df = jester_df[jester_df["rating"] != 99]

In [ ]:
profile_size = jester_df.groupby("user_id").size()
relevant_users = profile_size[profile_size >= 20].index.tolist()
jester_df = jester_df[jester_df["user_id"].isin(relevant_users)]

In [ ]:
print("Nr. of ratings: %d" % len(jester_df))
print("Nr. of users: %d" % jester_df["user_id"].nunique())
print("Nr. of items: %d" % jester_df["item_id"].nunique())
print("Density: %f" % (len(jester_df) / (jester_df["user_id"].nunique() * jester_df["item_id"].nunique())))
print("Avg. nr. of ratings per user: %f" % jester_df.groupby("user_id").size().mean())
print("Avg. nr. of ratings per item: %f" % jester_df.groupby("item_id").size().mean())

In [ ]:
user_sample = np.random.choice(jester_df["user_id"].unique(), size=5000, replace=False)
df = jester_df[jester_df["user_id"].isin(user_sample)]
print("Nr. of ratings: %d" % len(df))
print("Nr. of users: %d" % df["user_id"].nunique())
print("Nr. of items: %d" % df["item_id"].nunique())
print("Density: %f" % (len(df) / (df["user_id"].nunique() * df["item_id"].nunique())))
print("Avg. nr. of ratings per user: %f" % df.groupby("user_id").size().mean())
print("Avg. nr. of ratings per item: %f" % df.groupby("item_id").size().mean())

In [ ]:
df.to_csv("data/jester/sample.csv", sep=";", header=None, index=False)

In [ ]:
df.groupby("item_id").size().skew(), df.groupby("user_id").size().skew()

# Douban

In [ ]:
db_train_df = pd.read_csv("data/douban/db.train.rating", sep="\t", header=None, names=["user_id", "item_id", "rating"])
db_val_df = pd.read_csv("data/douban/db.valid.rating", sep="\t", header=None, names=["user_id", "item_id", "rating"])
db_test_df = pd.read_csv("data/douban/db.test.rating", sep="\t", header=None, names=["user_id", "item_id", "rating"])

douban_df = db_train_df.append(db_val_df).append(db_test_df)
douban_df.head()

In [ ]:
print("Nr. of ratings: %d" % len(douban_df))
print("Nr. of users: %d" % douban_df["user_id"].nunique())
print("Nr. of items: %d" % douban_df["item_id"].nunique())
print("Density: %f" % (len(douban_df) / (douban_df["user_id"].nunique() * douban_df["item_id"].nunique())))
print("Avg. nr. of ratings per user: %f" % douban_df.groupby("user_id").size().mean())
print("Avg. nr. of ratings per item: %f" % douban_df.groupby("item_id").size().mean())

In [ ]:
douban_df.to_csv("data/douban/douban.csv", sep=";", header=False, index=False)

# Ciao

In [ ]:
ciao_train_df = pd.read_csv("data/ciao/ciao.train.rating", sep="\t", header=None, names=["user_id", "item_id", "rating"])
ciao_val_df = pd.read_csv("data/ciao/ciao.valid.rating", sep="\t", header=None, names=["user_id", "item_id", "rating"])
ciao_test_df = pd.read_csv("data/ciao/ciao.test.rating", sep="\t", header=None, names=["user_id", "item_id", "rating"])

ciao_df = ciao_train_df.append(ciao_val_df).append(ciao_test_df)
ciao_df.head()

In [ ]:
print("Nr. of ratings: %d" % len(ciao_df))
print("Nr. of users: %d" % ciao_df["user_id"].nunique())
print("Nr. of items: %d" % ciao_df["item_id"].nunique())
print("Density: %f" % (len(ciao_df) / (ciao_df["user_id"].nunique() * ciao_df["item_id"].nunique())))
print("Avg. nr. of ratings per user: %f" % ciao_df.groupby("user_id").size().mean())
print("Avg. nr. of ratings per item: %f" % ciao_df.groupby("item_id").size().mean())

In [ ]:
ciao_df.to_csv("data/ciao/ciao.csv", sep=";", header=False, index=False)

# Goodreads

In [2]:
goodreads_df = pd.read_csv("data/goodreads/goodreads_interactions.csv", sep=",", usecols=["user_id", "book_id", "rating"])
goodreads_df.head()

,user_id,book_id,rating
0,0,948,5
1,0,947,5
2,0,946,5
3,0,945,5
4,0,944,5


In [3]:
goodreads_df.columns = ["user_id", "item_id", "rating"]

In [4]:
profile_size = goodreads_df.groupby("user_id").size()
relevant_users = profile_size[profile_size >= 20].index.tolist()
goodreads_df = goodreads_df[goodreads_df["user_id"].isin(relevant_users)]

In [5]:
print("Nr. of ratings: %d" % len(goodreads_df))
print("Nr. of users: %d" % goodreads_df["user_id"].nunique())
print("Nr. of items: %d" % goodreads_df["item_id"].nunique())
print("Density: %f" % (len(goodreads_df) / (goodreads_df["user_id"].nunique() * goodreads_df["item_id"].nunique())))
print("Avg. nr. of ratings per user: %f" % goodreads_df.groupby("user_id").size().mean())
print("Avg. nr. of ratings per item: %f" % goodreads_df.groupby("item_id").size().mean())

Nr. of ratings: 227545520
Nr. of users: 723208
Nr. of items: 2355940
Density: 0.000134
Avg. nr. of ratings per user: 314.633577
Avg. nr. of ratings per item: 96.583750


In [6]:
user_sample = np.random.choice(goodreads_df["user_id"].unique(), size=2500, replace=False)
df = goodreads_df[goodreads_df["user_id"].isin(user_sample)]
print("Nr. of ratings: %d" % len(df))
print("Nr. of users: %d" % df["user_id"].nunique())
print("Nr. of items: %d" % df["item_id"].nunique())
print("Density: %f" % (len(df) / (df["user_id"].nunique() * df["item_id"].nunique())))
print("Avg. nr. of ratings per user: %f" % df.groupby("user_id").size().mean())
print("Avg. nr. of ratings per item: %f" % df.groupby("item_id").size().mean())

Nr. of ratings: 731312
Nr. of users: 2500
Nr. of items: 256026
Density: 0.001143
Avg. nr. of ratings per user: 292.524800
Avg. nr. of ratings per item: 2.856397


In [ ]:
df.to_csv("data/goodreads/sample.csv", sep=";", header=None, index=False)

In [ ]:
df.groupby("item_id").size().skew(), df.groupby("user_id").size().skew()